In [1]:
import pandas as pd

Reading in assessment data for every TN Public school for 5 years of historical data

In [46]:
data_2017 = pd.read_csv('../data/school_based/2017.csv')
data_2018 =pd.read_csv('../data/school_based/2018.csv')
data_2019 = pd.read_csv('../data/school_based/2019.csv')
data_2021 = pd.read_csv('../data/school_based/2021.csv')
data_2022 = pd.read_csv('../data/school_based/2022.csv')

In [56]:
data_2017.head()

,year,system,system_name,school,school_name,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2017,10,Anderson County,2,Anderson County High School,Algebra I,10,All Students,50.0,**,**,**,**,**,**,**,**,**
1,2017,10,Anderson County,2,Anderson County High School,Algebra I,10,Economically Disadvantaged,25.0,**,**,**,**,**,**,**,**,**
2,2017,10,Anderson County,2,Anderson County High School,Algebra I,10,Non-Black/Hispanic/Native American,50.0,**,**,**,**,**,**,**,**,**
3,2017,10,Anderson County,2,Anderson County High School,Algebra I,10,Non-Economically Disadvantaged,25.0,**,**,**,**,**,**,**,**,**
4,2017,10,Anderson County,2,Anderson County High School,Algebra I,10,Non-English Learners,50.0,**,**,**,**,**,**,**,**,**


2017 does not have the school_name and system_name columns.  I will create a mapping from the 2018 dataframe to backfill the missing strings

In [47]:
# Create a unique mapping of 2018 dataframe
mapping_2018 = data_2018.drop_duplicates(subset=['system', 'school'])

# Merge the dataframes
data_2017 = pd.merge(data_2017, mapping_2018[['system', 'system_name', 'school', 'school_name']], on=['system', 'school'], how='left')

# Specify column order to match subsequent dataframes
new_order = ['year', 'system', 'system_name', 'school', 'school_name', 'subject', 'grade', 'subgroup', 'valid_tests',
             'n_below', 'n_approaching', 'n_on_track', 'n_mastered', 'pct_below', 
             'pct_approaching', 'pct_on_track', 'pct_mastered', 'pct_on_mastered']

# Reorder the columns
data_2017 = data_2017[new_order]

Looks like the mapping was successful, but did not compmpleteley solve the issue


In [50]:
data_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440633 entries, 0 to 440632
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   year             440633 non-null  int64  
 1   system           440633 non-null  int64  
 2   system_name      438204 non-null  object 
 3   school           440633 non-null  int64  
 4   school_name      438204 non-null  object 
 5   subject          440633 non-null  object 
 6   grade            440633 non-null  object 
 7   subgroup         440633 non-null  object 
 8   valid_tests      440633 non-null  float64
 9   n_below          440633 non-null  object 
 10  n_approaching    440633 non-null  object 
 11  n_on_track       440633 non-null  object 
 12  n_mastered       440633 non-null  object 
 13  pct_below        440633 non-null  object 
 14  pct_approaching  440633 non-null  object 
 15  pct_on_track     440633 non-null  object 
 16  pct_mastered     440633 non-null  obje

In [65]:
missing_2017 = data_2017[data_2017['system_name'].isna()]
missing_2017 = missing_2017[['system', 'school']]
missing_2017 = missing_2017.groupby('system')['school'].unique()
missing_2017

system
10            [105]
11              [0]
150             [0]
190      [425, 520]
231            [25]
300             [0]
470            [83]
580         [0, 75]
650             [0]
792    [2075, 2760]
794           [170]
800            [35]
820           [200]
830             [0]
860             [0]
985    [8035, 8080]
Name: school, dtype: object

In [54]:
data_2018.head(n=1)

,year,system,system_name,school,school_name,test,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,**,**,**,**,**,**,**,**,**


In [ ]:
data_2021.head(n=1)